In [1]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time
mixer.init()
sound = mixer.Sound('alarm.wav')


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pip install pygame

  Obtaining dependency information for pygame from https://files.pythonhosted.org/packages/82/61/93ae7afbd931a70510cfdf0a7bb0007540020b8d80bc1d8762ebdc46479b/pygame-2.5.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.8 MB 656.4 kB/s eta 0:00:17
    --------------------------------------- 0.1/10.8 MB 1.2 MB/s eta 0:00:09
   - -------------------------------------- 0.3/10.8 MB 1.5 MB/s eta 0:00:07
   - -------------------------------------- 0.4/10.8 MB 1.9 MB/s eta 0:00:06
   - -------------------------------------- 0.5/10.8 MB 1.9 MB/s eta 0:00:06
   -- ------------------------------------- 0.6/10.8 MB 2.0 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/10.8 MB 2.1 MB/s eta 0:00:05
   --- ------------------------------------ 0.8/10.8 MB 2.1 MB/s eta 0:00:05
   --- -----------------------------

In [3]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

mixer.init()
sound = mixer.Sound('alarm.wav')

face = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')

lbl=['Close','Open']

model = load_model("./resnet101.h5")
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]

while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2] 

    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        print("Right Eye Detected")
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2RGB)
        r_eye = cv2.resize(r_eye,(32,32))
        r_eye=  r_eye.reshape((-1, 32, 32, 3))

        # predict_classes is depricated
        #rpred = model.predict_classes(r_eye) 
        
        rpred = np.argmax(model.predict(r_eye), axis=-1)
        if(rpred[0]==1):
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        print("left Eye Detected")
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2RGB)  
        l_eye = cv2.resize(l_eye,(32,32))
        l_eye=l_eye.reshape((-1, 32, 32, 3))
        
        # predict_classes is depricated
        #lpred = model.predict_classes(l_eye)
        
        lpred = np.argmax(model.predict(l_eye), axis=-1)
        if(lpred[0]==1):
            lbl='Open'   
        if(lpred[0]==0):
            lbl='Closed'
        break

    if(rpred[0]==0 and lpred[0]==0):
        score=score+1
        cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # if(rpred[0]==1 or lpred[0]==1):
    else:
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        
    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>12):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            sound.play()
            
        except:  # isplaying = False
            pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

left Eye Detected
1/1 [==============================] - 4s 4s/step
left Eye Detected
1/1 [==============================] - 0s 97ms/step
left Eye Detected
1/1 [==============================] - 0s 102ms/step
Right Eye Detected
1/1 [==============================] - 0s 125ms/step
left Eye Detected
1/1 [==============================] - 0s 93ms/step
Right Eye Detected
1/1 [==============================] - 0s 119ms/step
left Eye Detected
1/1 [==============================] - 0s 108ms/step
left Eye Detected
1/1 [==============================] - 0s 110ms/step
left Eye Detected
1/1 [==============================] - 0s 109ms/step
left Eye Detected
1/1 [==============================] - 0s 105ms/step
Right Eye Detected
1/1 [==============================] - 0s 121ms/step
left Eye Detected
1/1 [==============================] - 0s 105ms/step
left Eye Detected
1/1 [==============================] - 0s 137ms/step
Right Eye Detected
1/1 [==============================] - 0s 106ms/step
left Ey

In [6]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

mixer.init()
sound = mixer.Sound('alarm.wav')

face = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')

lbl=['Close','Open']

model = load_model("resnet101.h5")
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]
while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2] 

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    closed_eye_detected = False  # Add a flag to track if closed eyes are detected

    for (x,y,w,h) in right_eye:
        print("Right Eye Detected")
        r_eye = frame[y:y+h,x:x+w]
        r_eye_gray = cv2.cvtColor(r_eye, cv2.COLOR_BGR2GRAY)
        r_eye_rgb = cv2.cvtColor(r_eye_gray, cv2.COLOR_GRAY2RGB)  # Convert to RGB

        r_eye_rgb = cv2.resize(r_eye_rgb, (32, 32))
        r_eye_rgb = r_eye_rgb.reshape((-1, 32, 32, 3))  # Reshape for prediction

        rpred = np.argmax(model.predict(r_eye_rgb), axis=-1)
        #if rpred[0] == 0:  # Closed eye detected
        #    closed_eye_detected = True
         #   break
        if(rpred[0]==1):
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        print("left Eye Detected")
        l_eye = frame[y:y+h,x:x+w]
        l_eye_gray = cv2.cvtColor(l_eye, cv2.COLOR_BGR2GRAY)
        l_eye_rgb = cv2.cvtColor(l_eye_gray, cv2.COLOR_GRAY2RGB)  # Convert to RGB

        l_eye_rgb = cv2.resize(l_eye_rgb, (32, 32))
        l_eye_rgb = l_eye_rgb.reshape((-1, 32, 32, 3))  # Reshape for prediction

        lpred = np.argmax(model.predict(l_eye_rgb), axis=-1)
        #if lpred[0] == 0:  # Closed eye detected
         #      closed_eye_detected = True
          #  break
        if(lpred[0]==1):
            lbl='Open'   
        if(lpred[0]==0):
            lbl='Closed'
        break

    if closed_eye_detected:
        score += 1  # Increment the score when closed eyes are detected
    else:
        score = 0  # Reset the score if no closed eyes are detected
        
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if score > 12:
        # Person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'), frame)
        try:
            sound.play()
        except:
            pass
        if thicc < 16:
            thicc += 2
        else:
            thicc -= 2
            if thicc < 2:
                thicc = 2
        cv2.rectangle(frame, (0,0), (width,height), (0,0,255), thicc)
        
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows() 


Right Eye Detected
1/1 [==============================] - 2s 2s/step
left Eye Detected
1/1 [==============================] - 0s 108ms/step
Right Eye Detected
1/1 [==============================] - 0s 122ms/step
left Eye Detected
1/1 [==============================] - 0s 113ms/step
Right Eye Detected
1/1 [==============================] - 0s 115ms/step
left Eye Detected
1/1 [==============================] - 0s 110ms/step
Right Eye Detected
1/1 [==============================] - 0s 109ms/step
left Eye Detected
1/1 [==============================] - 0s 115ms/step
Right Eye Detected
1/1 [==============================] - 0s 114ms/step
left Eye Detected
1/1 [==============================] - 0s 100ms/step
Right Eye Detected
1/1 [==============================] - 0s 110ms/step
left Eye Detected
1/1 [==============================] - 0s 126ms/step
Right Eye Detected
1/1 [==============================] - 0s 113ms/step
left Eye Detected
1/1 [==============================] - 0s 119ms/step
Ri

In [2]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

mixer.init()
sound = mixer.Sound('alarm.wav')

face = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')

lbl=['Close','Open']

model = load_model('./resnet101.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]

while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2] 

    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2RGB)
        r_eye = cv2.resize(r_eye,(32,32))
        r_eye=  r_eye.reshape((-1, 32, 32, 3))

        # predict_classes is depricated
        #rpred = model.predict_classes(r_eye) 
        
        rpred = np.argmax(model.predict(r_eye), axis=-1)
        if(rpred[0]==1):
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2RGB)  
        l_eye = cv2.resize(l_eye,(32,32))
        l_eye=l_eye.reshape((-1, 32, 32, 3))
        
        # predict_classes is depricated
        #lpred = model.predict_classes(l_eye)
        
        lpred = np.argmax(model.predict(l_eye), axis=-1)
        if(lpred[0]==1):
            lbl='Open'   
        if(lpred[0]==0):
            lbl='Closed'
        break

    if(rpred[0]==0 and lpred[0]==0):
        score=score+1
        cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # if(rpred[0]==1 or lpred[0]==1):
    else:
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        
    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>2):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            sound.play()
            
        except:  # isplaying = False
            pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



1/1 [==============================] - 0s 95ms/step
